In [1]:

import io
import os
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Arc
from IPython.display import display
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import mezzala
import itertools
import json
import urllib.request
from datetime import datetime  
from collections import defaultdict
from collections import Counter
from scipy import stats
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from PIL import Image
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import unidecode
import re




In [2]:
# Definir la ruta relativa desde la ubicación actual (notebooks) hasta la carpeta de datos
data_path = "../academyApp/data/archivos_parquet"

# Obtener la lista de archivos parquet que contienen "all"
parquet_files = [f for f in os.listdir(data_path) if f.endswith('.parquet') and 'all' in f.lower()]

# Procesar cada archivo y crear variables globales
for file in parquet_files:
    # Crear el nombre del DataFrame (reemplazar .parquet y agregar prefijo df_)
    df_name = 'df_' + file.replace('.parquet', '')
    
    # Leer el archivo parquet
    file_path = os.path.join(data_path, file)
    
    # Crear la variable global usando globals()
    globals()[df_name] = pd.read_parquet(file_path)
    
    # Imprimir información sobre el proceso
    print(f"Archivo procesado: {file}")
    print(f"Nombre del DataFrame: {df_name}")
    print(f"Dimensiones: {globals()[df_name].shape}")
    print("-" * 50)

Archivo procesado: leagues.parquet_all.parquet
Nombre del DataFrame: df_leagues_all
Dimensiones: (49, 7)
--------------------------------------------------
Archivo procesado: formations_league_all.parquet
Nombre del DataFrame: df_formations_league_all
Dimensiones: (34274, 9)
--------------------------------------------------
Archivo procesado: names_players_league_all.parquet
Nombre del DataFrame: df_names_players_league_all
Dimensiones: (17491, 12)
--------------------------------------------------
Archivo procesado: seasons_all.parquet
Nombre del DataFrame: df_seasons_all
Dimensiones: (112, 7)
--------------------------------------------------
Archivo procesado: sequence_data_league_all.parquet
Nombre del DataFrame: df_sequence_data_league_all
Dimensiones: (211193, 7)
--------------------------------------------------
Archivo procesado: players_league_all.parquet
Nombre del DataFrame: df_players_league_all
Dimensiones: (8980, 6)
--------------------------------------------------
Arch

In [3]:
df_teams_league_all

,team_id,name,name_en,iso_country_code,player_ids,season_id
0,7749,CD Getxo Cad. A,CD Getxo Cad. A,ES,[],3728
1,7757,Real Sociedad Cad. A,Real Sociedad Cad. A,ES,[],3728
2,7780,Athletic Club Cad. A,Athletic Bilbao Cad. A,ES,[],3728
3,7893,CD Kostkas Cad. A,CD Kostkas Cad. A,ES,[],3728
4,7897,Deportivo Alavés Cad. A,Deportivo Alavés U16,ES,"[251457, 251459, 366344, 366343, 366342, 36634...",3728
...,...,...,...,...,...,...
256,13797,Real Zaragoza Deportivo Aragón,"Real Zaragoza ""B""",ES,"[268010, 464356, 464357, 464358, 464359, 46436...",4019
257,15934,Utebo F.C.,Utebo FC,ES,"[329656, 329657, 329658, 329659, 329660, 32966...",4019
258,15936,CD Alfaro,CD Alfaro,ES,"[330959, 330960, 330961, 330962, 330963, 33096...",4019
259,16285,CD Anguiano,Anguiano,ES,"[541531, 541532, 541533, 541534, 541535, 54153...",4019


In [4]:
df_events_league_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719035 entries, 0 to 719034
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         719035 non-null  int64  
 1   event_period               719035 non-null  object 
 2   event_time                 718594 non-null  float64
 3   match_id                   719035 non-null  int64  
 4   team_id                    654221 non-null  float64
 5   player_id                  654221 non-null  float64
 6   x                          718594 non-null  float64
 7   y                          718594 non-null  float64
 8   attack_direction           719035 non-null  object 
 9   relative_event.id          285180 non-null  float64
 10  relative_event.event_time  285176 non-null  float64
 11  relative_event.player_id   220807 non-null  float64
 12  relative_event.x           285176 non-null  float64
 13  relative_event.y           28

In [ ]:

# Primero procesamos df_leagues_all como antes
leagues_expanded = df_leagues_all.copy()
leagues_expanded['season_ids'] = leagues_expanded['season_ids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
leagues_exploded = leagues_expanded.explode('season_ids').reset_index(drop=True)
leagues_exploded = leagues_exploded.rename(columns={'season_ids': 'season_id'})

# Merge con leagues
df_events_league_all = df_events_league_all.merge(
    leagues_exploded[['season_id', 'name']],
    on='season_id',
    how='left'
)
df_events_league_all = df_events_league_all.rename(columns={'name': 'Liga'})

# Merge con seasons
df_events_league_all = df_events_league_all.merge(
    df_seasons_all[['season_id', 'name']],
    on='season_id',
    how='left'
)
df_events_league_all = df_events_league_all.rename(columns={'name': 'Temporada'})

# Crear la columna de partido en df_matches_league_all
df_matches_league_all['partido'] = (
    df_matches_league_all['home_team_name'] + ' ' + 
    df_matches_league_all['home_team_score'].astype(str) + ' - ' + 
    df_matches_league_all['away_team_score'].astype(str) + ' ' + 
    df_matches_league_all['away_team_name']
)

# Merge con matches
df_events_league_all = df_events_league_all.merge(
    df_matches_league_all[['match_id', 'partido']],
    on='match_id',
    how='left'
)

# Merge con teams para añadir el nombre del equipo
df_events_league_all = df_events_league_all.merge(
    df_teams_league_all[['team_id', 'name']],
    on='team_id',
    how='left'
)
df_events_league_all = df_events_league_all.rename(columns={'name': 'equipo'})

# Verificamos el resultado
print("\nPrimeras 5 filas del DataFrame resultante:")
print(df_events_league_all[['match_id', 'Liga', 'Temporada', 'partido', 'equipo']].head())

print("\nEjemplos de equipos:")
print(df_events_league_all['equipo'].value_counts().head())

# Verificaciones de datos faltantes
print("\nVerificación de datos faltantes:")
for columna in ['Liga', 'Temporada', 'partido', 'equipo']:
    nulos = df_events_league_all[df_events_league_all[columna].isna()]
    if len(nulos) > 0:
        print(f"\nHay {len(nulos)} eventos sin {columna}")
        if columna == 'equipo':
            print("Team IDs sin correspondencia:", nulos['team_id'].unique())
        elif columna == 'partido':
            print("Match IDs sin correspondencia:", nulos['match_id'].unique())

# Mostrar dimensiones finales del DataFrame
print("\nDimensiones finales del DataFrame:", df_events_league_all.shape)

In [ ]:
# Para archivos Parquet, verificamos primero que el DataFrame se haya cargado correctamente
if df_events_league_all is not None:
    # Identificamos columnas completamente nulas
    columnas_nulas = df_events_league_all.columns[df_events_league_all.isna().all()].tolist()
    
    # Identificamos columnas que son completamente cero
    # Usamos .astype(float).eq(0) para manejar diferentes tipos de datos numéricos en Parquet
    columnas_cero = df_events_league_all.select_dtypes(include=['number']).columns[
        df_events_league_all.select_dtypes(include=['number']).eq(0).all()
    ].tolist()
    
    # Combinamos ambas listas
    columnas_nulas_o_cero = list(set(columnas_nulas + columnas_cero))
    
    # Imprimimos los resultados
    if columnas_nulas_o_cero:
        print("Las siguientes columnas son completamente nulas o cero:")
        for columna in columnas_nulas_o_cero:
            if columna in columnas_nulas and columna in columnas_cero:
                print(f"{columna} (nula y cero)")
            elif columna in columnas_nulas:
                print(f"{columna} (nula)")
            else:
                print(f"{columna} (cero)")
    else:
        print("No hay columnas que sean completamente nulas o cero.")
else:
    print("Error: El DataFrame no se ha cargado correctamente desde el archivo Parquet.")

In [ ]:
# Lista de columnas a eliminar
columnas_a_eliminar = [
    'extra_match_result', 'venue', 'birth_date', 'player_role', 'venue_ground_height', 
    'extra_full_time', 'live_analysing', 'player_last_name', 'player_name', 'root_player_id',
    'home_team_id', 'home_team_name', 'home_team_iso_country_code', 
    'away_team_id', 'away_team_name', 'away_team_iso_country_code',
    'venue_id', 'venue_display_name', 'venue_ground_width', 'season_group_name', 
    'season_id_player', 'team_id_player', 'analysis_completed_datetime', 'is_analysis_finished', 'season_id','round_id','name','player_ids', 'iso_country_code'
]

# Eliminar las columnas
df_events_league_all = df_events_league_all.drop(columns=columnas_a_eliminar)

# Renombrar 'season_id_match' a 'season_id'
df_events_league_all = df_events_league_all.rename(columns={'season_id_match': 'season_id'})

# Cambiar la columna del tiempo a minutos y segundos
def ms_to_min_sec(ms):
    if pd.isna(ms):
        return "00:00"
    total_seconds = int(ms // 1000)
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    return f"{int(minutes):02d}:{int(seconds):02d}"

# Convertir event_time a minutos:segundos
df_events_league_all['event_time'] = df_events_league_all['event_time'].apply(ms_to_min_sec)

# Crear la nueva columna 'match'
df_events_league_all['match'] = (
    df_events_league_all['home_team_name_en'] + ' ' +
    df_events_league_all['home_team_score'].astype(str) + '-' +
    df_events_league_all['away_team_score'].astype(str) + ' ' +
    df_events_league_all['away_team_name_en']
)

# Combinar las columnas y aplicar el formato deseado para 'player'
df_events_league_all['player'] = (df_events_league_all['player_name_en'].str.capitalize() + ' ' + 
                                  df_events_league_all['player_last_name_en'].str.capitalize())

# Aplicar el formato de título a toda la cadena
df_events_league_all['player'] = df_events_league_all['player'].str.title()

# Mostrar las primeras filas para verificar
print(df_events_league_all[['player_name_en', 'player_last_name_en', 'player']].head())

# Verificar el número total de jugadores únicos
print(f"\nNúmero total de jugadores únicos: {df_events_league_all['player'].nunique()}")

# Eliminar las columnas originales utilizadas para crear 'match' y 'player'
df_events_league_all = df_events_league_all.drop(columns=['home_team_name_en', 'home_team_score', 'away_team_score', 'away_team_name_en', 'player_name_en', 'player_last_name_en'])

# Mostrar las primeras filas del DataFrame resultante
print("\nPrimeras filas del DataFrame resultante:")
print(df_events_league_all.head())

# Mostrar información sobre las columnas restantes
print("\nColumnas restantes en el DataFrame:")
print(df_events_league_all.columns.tolist())

# Mostrar la forma del DataFrame después de las modificaciones
print(f"\nForma del DataFrame después de las modificaciones: {df_events_league_all.shape}")

In [ ]:
# Borrar todas las filas con los valores nulos en eventType y player_id
df_events_league_all = df_events_league_all.dropna(subset=['eventType', 'player_id', 'player'])


In [ ]:
# Diccionario de traducción
traduccion = {
    # EventType
    'Pass': 'Pase',
    'Pass Received': 'Pase Recibido',
    'Clearance': 'Despeje',
    'Duel': 'Duelo',
    'Intervention': 'Intervención',
    'Foul Won': 'Falta Recibida',
    'Foul': 'Falta',
    'Shot': 'Disparo',
    'Save': 'Atajada',
    'Tackle': 'Entrada',
    'Offside': 'Fuera de Juego',
    'Block': 'Bloqueo',
    'Pause': 'Pausa',
    'Ball Received': 'Balón Recibido',
    'Interception': 'Intercepción',
    'Recovery': 'Recuperación',
    'Aerial Clearance': 'Despeje Aereo',
    'Take-On': 'Regate',
    'Deflection': 'Desvío',
    'Hit': 'Golpeo',
    'Carry': 'Conduccion',
    'Control Under Pressure': 'Control Bajo Presión',
    'Defensive Line Support': 'Apoyo a la Línea Defensiva',
    'Error': 'Error',
    'Goal Conceded': 'Gol Concedido',
    'Substitution': 'Sustitución',
    'Cross Received': 'Centro Recibido',
    'Own Goal': 'Gol en Propia',
    'Set Piece': 'Jugada a Balón Parado',

    # Outcome
    'Successful': 'Exitoso',
    'Unsuccessful': 'Fallido',
    'On Target': 'A Puerta',
    'Off Target': 'Fuera',
    'Blocked': 'Bloqueado',
    'Goal': 'Gol',
    'Low Quality Shot': 'Disparo de Baja Calidad',
    'Keeper Rush-Out': 'Salida del Portero',

    # Cross
    'True': 'Sí',
    'False': 'No',

    # KeyPass
    'True': 'Sí',
    'False': 'No',

    # Assist
    'True': 'Sí',
    'False': 'No',

    # SubEventType
    'Ground': 'Suelo',
    'Loose Ball': 'Balón Suelto',
    'Aerial': 'Aéreo',
    'Foul': 'Falta',
    'Yellow Card': 'Tarjeta Amarilla',
    'Handball Foul': 'Falta por Mano',
    'Foul Throw': 'Saque de Banda Incorrecto',
    'Red Card': 'Tarjeta Roja',
    'Catch': 'Atrapada',
    'Parry': 'Despeje',
    'Off': 'Sale',
    'On': 'Entra',
    'Goal Kick': 'Saque de Puerta',

    # BodyPart
    'Head': 'Cabeza',
    'Left Foot': 'Pie Izquierdo',
    'Right Foot': 'Pie Derecho',
    'Other': 'Otro',
    'Upper Body': 'Parte Superior del Cuerpo',
    'Hands': 'Manos',
    'Lower Body': 'Parte Inferior del Cuerpo'
}

# Función para traducir valores
def traducir(valor):
    return traduccion.get(valor, valor)

# Aplicar la traducción a las columnas relevantes
columnas_a_traducir = ['eventType', 'outcome', 'cross', 'keyPass', 'assist', 'subEventType', 'bodyPart']

# Renombrar columnas
df_events_league_all = df_events_league_all.rename(columns={
    'x': 'xstart',
    'y': 'ystart',
    'relative_event.x': 'xend',
    'relative_event.y': 'yend'
})


for columna in columnas_a_traducir:
    df_events_league_all[columna] = df_events_league_all[columna].apply(traducir)

# Renombrar 'name_en' a 'team_name'
df_events_league_all = df_events_league_all.rename(columns={'name_en': 'team_name'})

# Mostrar las primeras filas del DataFrame traducido
print(df_events_league_all[columnas_a_traducir].head())

# Verificar los valores únicos traducidos en cada columna
for columna in columnas_a_traducir:
    print(f"\nValores únicos en {columna}:")
    print(df_events_league_all[columna].unique())

In [ ]:
# Traducir columnas relevantes
df_events_league_all = df_events_league_all.rename(columns={
    'eventType': 'tipo_evento',
    'outcome': 'resultado',
    'cross': 'centro',
    'keyPass': 'pase_clave',
    'assist': 'asistencia',
    'subEventType': 'subtipo_evento',
    'bodyPart': 'parte_cuerpo'
})

In [ ]:
# Parametrizar las coordenadas de 0-1 a 0-100
df_events_league_all['xstart'] = df_events_league_all['xstart'] * 100
df_events_league_all['ystart'] = df_events_league_all['ystart'] * 100
df_events_league_all['xend'] = df_events_league_all['xend'] * 100
df_events_league_all['yend'] = df_events_league_all['yend'] * 100

def determinar_zona(x, y):  # Usamos nombres genéricos x e y
    if pd.isna(x) or pd.isna(y):
        return 'Unknown'
    if 0 <= y < 16 and 0 <= x < 19:
        return '1'
    elif 0 <= y < 16 and 19 <= x < 81:
        return '2'
    elif 0 <= y < 16 and 81 <= x <= 100:
        return '3'
    elif 16 <= y < 33.3 and 0 <= x < 19:
        return '4'
    elif 16 <= y < 33.3 and 19 <= x < 81:
        return '5'
    elif 16 <= y <= 33.3 and 81 <= x <= 100:
        return '6'
    elif 33.3 <= y < 50 and 0 <= x < 19:
        return '7'
    elif 33.3 <= y < 50 and 19 <= x < 81:
        return '8'
    elif 33.3 <= y <= 50 and 81 <= x <= 100:
        return '9'
    elif 50 <= y < 66.6 and 0 <= x < 19:
        return '10'
    elif 50 <= y < 66.6 and 19 <= x < 81:
        return '11'
    elif 50 <= y < 66.6 and 81 <= x <= 100:
        return '12'
    elif 66.6 <= y < 83.3 and 0 <= x < 19:
        return '13'
    elif 66.6 <= y < 83.3 and 19 <= x < 81:
        return '14'
    elif 66.6 <= y < 83.3 and 81 <= x <= 100:
        return '15'
    elif 83.3 <= y <= 100 and 0 <= x < 19:
        return '16'
    elif 83.3 <= y <= 100 and 19 <= x < 81:
        return '17'
    elif 83.3 <= y <= 100 and 81 <= x <= 100:
        return '18'
    else:
        return 'Unknown'

# Aplicar la función a las columnas de inicio y fin
df_events_league_all['zona_inicio'] = df_events_league_all.apply(lambda row: determinar_zona(row['xstart'], row['ystart']), axis=1)
df_events_league_all['zona_fin'] = df_events_league_all.apply(lambda row: determinar_zona(row['xend'], row['yend']), axis=1)

# Mostrar las primeras filas para verificar
print(df_events_league_all[['xstart', 'ystart', 'xend', 'yend', 'zona_inicio', 'zona_fin']].head())

# Contar la frecuencia de cada zona
print("\nFrecuencia de zonas de inicio:")
print(df_events_league_all['zona_inicio'].value_counts())

print("\nFrecuencia de zonas de fin:")
print(df_events_league_all['zona_fin'].value_counts())

In [ ]:


# Función auxiliar para convertir los event_ids a lista
def convert_to_list(value):
    if isinstance(value, np.ndarray):
        return value.tolist()
    if isinstance(value, str):
        numbers = re.findall(r'\d+', value)
        return [int(x) for x in numbers]
    if isinstance(value, list):
        return value
    try:
        numbers = re.findall(r'\d+', str(value))
        return [int(x) for x in numbers]
    except:
        return []

# Crear un diccionario para mapear id_secuencia a event_ids
sequence_events_map = {}
for index, row in df_sequence_data_league_all.iterrows():
    event_ids = convert_to_list(row['event_ids'])
    sequence_events_map[index + 1] = event_ids

# Crear un diccionario para mapear event_id a id_secuencia
sequence_map = {}
for index, row in df_sequence_data_league_all.iterrows():
    event_ids = convert_to_list(row['event_ids'])
    for event_id in event_ids:
        sequence_map[event_id] = index + 1

# Crear la columna id_secuencia en df_events_league_all
df_events_league_all['id_secuencia'] = df_events_league_all['event_id'].map(sequence_map)

# Función para obtener eventos y zonas en orden
def get_ordered_events_and_zones(group):
    ordered_events = []
    ordered_zones = []
    event_ids = sequence_events_map.get(group.name, [])
    for event_id in event_ids:
        event = group[group['event_id'] == event_id]
        if not event.empty:
            ordered_events.append(str(event['tipo_evento'].iloc[0]))
            ordered_zones.append(str(event['zona_inicio'].iloc[0]))
    return pd.Series({
        'eventos_secuencia': ', '.join(ordered_events) if ordered_events else None,
        'zonas_secuencia': ', '.join(ordered_zones) if ordered_zones else None,
        'events_id_secuencia': ', '.join(map(str, event_ids)) if event_ids else None
    })

# Aplicar la función a los grupos
result = df_events_league_all.groupby('id_secuencia').apply(get_ordered_events_and_zones)

# Unir los resultados al dataframe original, manteniendo todas las filas
df_events_league_all = df_events_league_all.merge(result, on='id_secuencia', how='left')

# Llenar los valores NaN con cadenas vacías para mantener la consistencia
df_events_league_all['eventos_secuencia'] = df_events_league_all['eventos_secuencia'].fillna('')
df_events_league_all['zonas_secuencia'] = df_events_league_all['zonas_secuencia'].fillna('')
df_events_league_all['events_id_secuencia'] = df_events_league_all['events_id_secuencia'].fillna('')

# Mostrar las primeras filas para verificar
print(df_events_league_all[['event_id', 'id_secuencia', 'tipo_evento', 'zona_inicio', 
                           'eventos_secuencia', 'zonas_secuencia', 'events_id_secuencia']].head(10))

# Información adicional
print("\nNúmero de secuencias únicas:", df_events_league_all['id_secuencia'].nunique())

print("\nEjemplos de eventos_secuencia:")
print(df_events_league_all['eventos_secuencia'].value_counts().head())

print("\nEjemplos de zonas_secuencia:")
print(df_events_league_all['zonas_secuencia'].value_counts().head())

# Opcional: Mostrar la distribución de longitudes de secuencias
df_events_league_all['longitud_secuencia'] = df_events_league_all['eventos_secuencia'].str.count(',') + 1
df_events_league_all.loc[df_events_league_all['eventos_secuencia'] == '', 'longitud_secuencia'] = 0
print("\nDistribución de longitudes de secuencias:")
print(df_events_league_all['longitud_secuencia'].value_counts().sort_index().head())

# Mostrar algunos ejemplos de events_id_secuencia
print("\nEjemplos de events_id_secuencia:")
print(df_events_league_all['events_id_secuencia'].head())

In [ ]:
# Traducir la columna event_period
df_events_league_all['periodo'] = df_events_league_all['event_period'].map({
    'FIRST_HALF': '1a_parte',
    'SECOND_HALF': '2a_parte'
})

# Función para determinar el tramo del partido
def determinar_tramo(row):
    periodo = row['periodo']
    tiempo = row['event_time']
    
    if pd.isna(tiempo):
                                return 'Desconocido'
    
    minutos = int(tiempo.split(':')[0])
    
    if periodo == '1a_parte':
        if 0 <= minutos <= 15:
                                return '0-15\''
        elif 15 < minutos <= 30:
                                return '15-30\''
        else:
                                return '30-45\''
    elif periodo == '2a_parte':
        if 45 <= minutos <= 60:
                                return '45-60\''
        elif 60 < minutos <= 75:
                                return '60-75\''
        else:
                                return '75-90\''
    else:
                                return 'Desconocido'

# Aplicar la función para crear la columna tramo_partido
df_events_league_all['tramo_partido'] = df_events_league_all.apply(determinar_tramo, axis=1)

# Borrar la columna event_period
df_events_league_all = df_events_league_all.drop(columns=['event_period'])

# Mostrar las primeras filas para verificar
print(df_events_league_all[['periodo', 'event_time', 'tramo_partido']].head(10))

# Información adicional
print("\nDistribución de períodos:")
print(df_events_league_all['periodo'].value_counts())

print("\nDistribución de tramos del partido:")
print(df_events_league_all['tramo_partido'].value_counts())

# Verificar si hay algún valor inesperado
print("\nValores únicos en 'periodo':")
print(df_events_league_all['periodo'].unique())

print("\nValores únicos en 'tramo_partido':")
print(df_events_league_all['tramo_partido'].unique())

# Verificar que event_period ya no está en el DataFrame
print("\nColumnas en el DataFrame:")
print(df_events_league_all.columns.tolist())

In [ ]:
def calcular_angulo(row):
    dx = row['xend'] - row['xstart']
    dy = row['yend'] - row['ystart']
    
    if dx == 0 and dy == 0:
        return 0  # No hay movimiento
    
    angulo_rad = np.arctan2(dy, -dx)  # Negamos dx para invertir el eje x
    angulo_deg = np.degrees(angulo_rad)
    
    # Asegurarse de que el ángulo esté en el rango [0, 360)
    angulo_deg = angulo_deg % 360
    
    return round(angulo_deg, 2)

df_events_league_all['angulo_pase'] = df_events_league_all.apply(calcular_angulo, axis=1)

# Imprimir ejemplos aleatorios
num_ejemplos = 10
ejemplos = df_events_league_all.sample(n=num_ejemplos)
print(ejemplos[['xstart', 'ystart', 'xend', 'yend', 'angulo_pase']])

In [ ]:

def safe_int(x):
    try:
        return int(x)
    except ValueError:
        return -1

def clasificar_pase_por_angulo(angulo):
    if 60 <= angulo <= 120:
        return 'adelante'
    elif 210 <= angulo <= 300:
        return 'atras'
    else:
        return 'horizontal'

def clasificar_zona(zona_inicio):
    zona = safe_int(zona_inicio)
    if zona < 7:
        return 'inicio'
    elif 7 <= zona < 13:
        return 'creacion'
    else:
        return 'finalizacion'

def calcular_metricas(df):
    # pases_recibidos_perfilado
    df['pases_recibidos_perfilado'] = 0

    # Función auxiliar para verificar el siguiente evento
    def verificar_siguiente_evento(eventos):
        eventos_list = eventos.split(',')
        if 'Pase Recibido' in eventos_list:
            index = eventos_list.index('Pase Recibido')
            if index + 1 < len(eventos_list):
                siguiente_evento = eventos_list[index + 1].strip()
                return siguiente_evento in ['Pase', 'Regate', 'Conduccion']
        return False

    # Aplicar la función a cada fila
    df['pases_recibidos_perfilado'] = df.apply(
        lambda row: 1 if row['tipo_evento'] == 'Pase Recibido' and verificar_siguiente_evento(row['eventos_secuencia']) else 0,
        axis=1
    )

    # Conducciones por carril
    df['conducciones_carril_izquierdo'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['ystart'].between(0, 33.33)),
        1, 0
    )
    df['conducciones_carril_central'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['ystart'].between(33.33, 66.66)),
        1, 0
    )
    df['conducciones_carril_derecho'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['ystart'] > 66.66),
        1, 0
    )

    # Despejes por zona
    df['despejes_zona_area'] = np.where(
        (df['tipo_evento'] == 'Despeje') & (df['zona_inicio'] == '2'),
        1, 0
    )
    df['despejes_zona_baja'] = np.where(
        (df['tipo_evento'] == 'Despeje') & (df['zona_inicio'].isin(['1', '3', '4', '5', '6'])),
        1, 0
    )
    df['despejes_zona_media'] = np.where(
        (df['tipo_evento'] == 'Despeje') & (df['zona_inicio'].apply(safe_int).between(7, 12)),
        1, 0
    )
    df['despejes_zona_alta'] = np.where(
        (df['tipo_evento'] == 'Despeje') & (df['zona_inicio'].apply(safe_int) > 12),
        1, 0
    )

    # Pases Progresivos
    condicion_base1 = (
        (df['tipo_evento'] == 'Pase') &
        (df['angulo_pase'].between(60, 120, inclusive='both')) &
        (df['zona_fin'].apply(safe_int) > df['zona_inicio'].apply(safe_int))
    )
    condicion_base2 = (
        (df['tipo_evento'] == 'Pase') &
        (df['eventos_secuencia'].str.split(',').str[-1].isin(['Disparo', 'Regate']))
    )
    condicion_base3 = (
        (df['tipo_evento'] == 'Pase') &
        (df['zonas_secuencia'].str.split(',').str[-1].isin(['16', '17', '18']))
    )
    condicion_base4 = (
        (df['tipo_evento'] == 'Pase') &
        (df['ystart'] < df['yend']) &
        (df['zona_fin'].isin(['13', '14', '15']))
    )
    condicion_pase_progresivo = condicion_base1 | condicion_base2 | condicion_base3 | condicion_base4
    df['pases_progresivos_inicio'] = np.where(
        (df['zona_inicio'].apply(safe_int) < 7) & condicion_pase_progresivo,
        np.where(df['resultado'] == 'Exitoso', 1, 0), 0
    )
    df['pases_progresivos_creacion'] = np.where(
        (df['zona_inicio'].apply(safe_int).between(7, 12)) & condicion_pase_progresivo,
        np.where(df['resultado'] == 'Exitoso', 1, 0), 0
    )
    df['pases_progresivos_finalizacion'] = np.where(
        (df['zona_inicio'].apply(safe_int) > 12) & condicion_pase_progresivo,
        np.where(df['resultado'] == 'Exitoso', 1, 0), 0
    )

    # Conducciones por zona
    df['conducciones_inicio'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['zona_inicio'].apply(safe_int) < 7), 1, 0
    )
    df['conducciones_creacion'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['zona_inicio'].apply(safe_int).between(7, 12)), 1, 0
    )
    df['conducciones_finalizacion'] = np.where(
        (df['tipo_evento'] == 'Conduccion') & (df['zona_inicio'].apply(safe_int) > 12), 1, 0
    )
 
    # Pases fallidos
    df['pases_fallidos_campo_propio'] = np.where(
        (df['tipo_evento'] == 'Pase') & 
        (df['zona_inicio'].apply(safe_int).between(1, 9)) &
        (df['zona_fin'].apply(safe_int).between(1, 9)) &
        (df['resultado'] != 'Exitoso'),
        1, 0)

    df['pases_fallidos_campo_contrario'] = np.where(
        (df['tipo_evento'] == 'Pase') & 
        (df['zona_inicio'].apply(safe_int) > 9) &
        (df['zona_fin'].apply(safe_int) > 9) &
        (df['resultado'] != 'Exitoso'),
        1, 0)
    
# Pases por dirección y zona
    for direccion in ['adelante', 'atras', 'horizontal']:
        for zona in ['inicio', 'creacion', 'finalizacion']:
            column_name = f'pases_{direccion}_{zona}'
            df[column_name] = 0

    mask_pase = df['tipo_evento'] == 'Pase'
    df.loc[mask_pase, 'direccion_pase'] = df.loc[mask_pase, 'angulo_pase'].apply(clasificar_pase_por_angulo)
    df.loc[mask_pase, 'zona_pase'] = df.loc[mask_pase, 'zona_inicio'].apply(clasificar_zona)

    for direccion in ['adelante', 'atras', 'horizontal']:
        for zona in ['inicio', 'creacion', 'finalizacion']:
            column_name = f'pases_{direccion}_{zona}'
            mask = ((df['tipo_evento'] == 'Pase') & 
                   (df['direccion_pase'] == direccion) & 
                   (df['zona_pase'] == zona))
            df.loc[mask, column_name] = 1

    # Pases totales por dirección
    df['pases_adelante'] = df['pases_adelante_inicio'] + df['pases_adelante_creacion'] + df['pases_adelante_finalizacion']
    df['pases_atras'] = df['pases_atras_inicio'] + df['pases_atras_creacion'] + df['pases_atras_finalizacion']
    df['pases_horizontal'] = df['pases_horizontal_inicio'] + df['pases_horizontal_creacion'] + df['pases_horizontal_finalizacion']
    
    # Cambios de Orientación
    df['cambios_orientacion_exitosos'] = np.where(
    (df['tipo_evento'] == 'Pase') &
    (((df['xstart'] > 60) & (df['zona_fin'].isin(['1', '4', '7', '10', '13', '16']))) |
     ((df['xstart'] < 40) & (df['zona_fin'].isin(['3', '6', '9', '12', '15', '18'])))) &
    (df['resultado'] == 'Exitoso'),
    1, 0
    )
    df['cambios_orientacion_fallidos'] = np.where(
    (df['tipo_evento'] == 'Pase') &
    (((df['xstart'] > 60) & (df['zona_fin'].isin(['1', '4', '7', '10', '13', '16']))) |
     ((df['xstart'] < 40) & (df['zona_fin'].isin(['3', '6', '9', '12', '15', '18'])))) &
    (df['resultado'] == 'Fallido'),
    1, 0
    )

    # Disparos
    df['disparos_a_puerta'] = np.where(
        (df['tipo_evento'] == 'Disparo') & (df['resultado'] == 'A Puerta'),
        1, 0
    )
    df['disparos_fuera'] = np.where(
        (df['tipo_evento'] == 'Disparo') & (df['resultado'] == 'Fuera'),
        1, 0
    )

    # Regates
    df['regates_zona_inicio'] = np.where(
        (df['tipo_evento'] == 'Regate') & (df['zona_inicio'].apply(safe_int) < 7),
        1, 0
    )
    df['regates_zona_creacion'] = np.where(
        (df['tipo_evento'] == 'Regate') & (df['zona_inicio'].apply(safe_int).between(7, 12)),
        1, 0
    )
    df['regates_zona_finalizacion'] = np.where(
        (df['tipo_evento'] == 'Regate') & (df['zona_inicio'].apply(safe_int) > 12),
        1, 0
    )

    # Duelos
    for tipo in ['aereos', 'suelo']:
        for resultado in ['ganados', 'perdidos']:
            for zona in ['area', 'baja', 'media', 'alta']:
                column_name = f'duelos_{tipo}_{resultado}_zona_{zona}'
                zona_condition = (
                    (df['zona_inicio'] == '2') if zona == 'area' else
                    (df['zona_inicio'].apply(safe_int) < 7) if zona == 'baja' else
                    (df['zona_inicio'].apply(safe_int).between(7, 12)) if zona == 'media' else
                    (df['zona_inicio'].apply(safe_int) > 12)
                )
                df[column_name] = np.where(
                    (df['tipo_evento'] == 'Duelo') &
                    (df['subtipo_evento'] == ('Aéreo' if tipo == 'aereos' else 'Balón Suelto')) &
                    (df['resultado'] == ('Exitoso' if resultado == 'ganados' else 'Fallido')) &
                    zona_condition,
                    1, 0
                )

# Intercepciones
    for zona in ['area', 'baja', 'media', 'alta']:
        column_name = f'intercepciones_zona_{zona}'
        zona_condition = (
            (df['zona_inicio'] == '2') if zona == 'area' else
            (df['zona_inicio'].apply(safe_int) < 7) if zona == 'baja' else
            (df['zona_inicio'].apply(safe_int).between(7, 12)) if zona == 'media' else
            (df['zona_inicio'].apply(safe_int) > 12)
        )
        df[column_name] = np.where(
            (df['tipo_evento'] == 'Intercepción') &
            zona_condition,
            1, 0
        )

    # Recuperaciones
    for zona in ['area', 'baja', 'media', 'alta']:
        column_name = f'recuperaciones_zona_{zona}'
        zona_condition = (
            (df['zona_inicio'] == '2') if zona == 'area' else
            (df['zona_inicio'].apply(safe_int) < 7) if zona == 'baja' else
            (df['zona_inicio'].apply(safe_int).between(7, 12)) if zona == 'media' else
            (df['zona_inicio'].apply(safe_int) > 12)
        )
        df[column_name] = np.where(
            (df['tipo_evento'] == 'Recuperación') &
            zona_condition,
            1, 0
        )

    # Entradas
    for resultado in ['ganadas', 'perdidas']:
        for zona in ['area', 'baja', 'media', 'alta']:
            column_name = f'entradas_{resultado}_zona_{zona}'
            zona_condition = (
                (df['zona_inicio'] == '2') if zona == 'area' else
                (df['zona_inicio'].apply(safe_int) < 7) if zona == 'baja' else
                (df['zona_inicio'].apply(safe_int).between(7, 12)) if zona == 'media' else
                (df['zona_inicio'].apply(safe_int) > 12)
            )
            df[column_name] = np.where(
                (df['tipo_evento'] == 'Entrada') &
                (df['resultado'] == ('Exitoso' if resultado == 'ganadas' else 'Fallido')) &
                zona_condition,
                1, 0
            )

    # Faltas
    for zona in ['area', 'baja', 'media', 'alta']:
        column_name = f'faltas_zona_{zona}'
        zona_condition = (
            (df['zona_inicio'] == '2') if zona == 'area' else
            (df['zona_inicio'].apply(safe_int) < 7) if zona == 'baja' else
            (df['zona_inicio'].apply(safe_int).between(7, 12)) if zona == 'media' else
            (df['zona_inicio'].apply(safe_int) > 12)
        )
        df[column_name] = np.where(
            (df['tipo_evento'] == 'Falta') &
            (df['subtipo_evento'] == 'Falta') &
            zona_condition,
            1, 0
        )

    # Tarjetas y amonestaciones
    df['tarjetas_amarillas'] = np.where(
        (df['tipo_evento'] == 'Falta') & (df['subtipo_evento'] == 'Tarjeta Amarilla'), 1, 0
    )
    df['tarjetas_rojas'] = np.where(
        (df['tipo_evento'] == 'Falta') & (df['subtipo_evento'] == 'Tarjeta Roja'), 1, 0
    )
    df['amonestado'] = df.groupby(['match_id', 'player_id'])['tarjetas_amarillas'].cumsum().gt(0).astype(int)

    # Otras métricas
    df['faltas_recibidas'] = np.where(df['tipo_evento'] == 'Falta Recibida', 1, 0)
    df['fueras_de_juego'] = np.where(df['tipo_evento'] == 'Fuera de Juego', 1, 0)

    # Pases largos
    df['pases_largos_exitosos'] = np.where(
       (df['tipo_evento'] == 'Pase') &
       (df['resultado'] == 'Exitoso') &
       ((df['zona_fin'].apply(safe_int) - df['zona_inicio'].apply(safe_int)) > 7),
    1, 0
    )

    df['pases_largos_fallidos'] = np.where(
        (df['tipo_evento'] == 'Pase') &
        (df['resultado'] == 'Fallido') &
        (df['zona_inicio'].isin(['1', '2', '3'])) &
        ((df['zona_fin'].apply(safe_int) - df['zona_inicio'].apply(safe_int)) > 7),
        1, 0
    )

    # Centros
    df['centros_alejados'] = np.where(
        (df['tipo_evento'] == 'Pase') &
        (df['centro'] == True) &
        (df['zona_inicio'].apply(safe_int) < 16),
        1, 0
    )

    df['centros_linea_fondo'] = np.where(
        (df['tipo_evento'] == 'Pase') &
        (df['centro'] == True) &
        (df['zona_inicio'].isin(['16', '18'])),
        1, 0
    )

    # Pases exitosos por campo
    df['pases_exitosos_campo_propio'] = np.where(
        (df['tipo_evento'] == 'Pase') & 
        (df['zona_inicio'].apply(safe_int).between(1, 9)) &
        (df['zona_fin'].apply(safe_int).between(1, 9)) &
        (df['resultado'] == 'Exitoso'),
        1, 0)

    df['pases_exitosos_campo_contrario'] = np.where(
        (df['tipo_evento'] == 'Pase') & 
        (df['zona_inicio'].apply(safe_int) > 9) &
        (df['zona_fin'].apply(safe_int) > 9) &
        (df['resultado'] == 'Exitoso'),
        1, 0)

    return df

def main(df_events_league_all):
    df_eventos_metricas = calcular_metricas(df_events_league_all)
    
    # Lista de todas las métricas calculadas
    metricas = [
        # Pases
        'pases_adelante', 'pases_atras', 'pases_horizontal',
        'pases_adelante_inicio', 'pases_adelante_creacion', 'pases_adelante_finalizacion',
        'pases_atras_inicio', 'pases_atras_creacion', 'pases_atras_finalizacion',
        'pases_horizontal_inicio', 'pases_horizontal_creacion', 'pases_horizontal_finalizacion',
        'pases_progresivos_inicio', 'pases_progresivos_creacion', 'pases_progresivos_finalizacion',
        'pases_fallidos_campo_propio', 'pases_fallidos_campo_contrario',
        'pases_exitosos_campo_propio', 'pases_exitosos_campo_propio',
        'pases_largos_exitosos', 'pases_largos_fallidos',
        'centros_alejados', 'centros_linea_fondo',
        'cambios_orientacion_exitosos', 'cambios_orientacion_fallidos',
        
        # Recepciones
        'pases_recibidos_perfilado',
        
        # Conducciones
        'conducciones_inicio', 'conducciones_creacion', 'conducciones_finalizacion',
        'conducciones_carril_izquierdo', 'conducciones_carril_central', 'conducciones_carril_derecho',
        
        # Regates
        'regates_zona_inicio', 'regates_zona_creacion', 'regates_zona_finalizacion',
        
        # Disparos
        'disparos_a_puerta', 'disparos_fuera',
        
        # Despejes
        'despejes_zona_area', 'despejes_zona_baja', 'despejes_zona_media', 'despejes_zona_alta',
        
        # Duelos
        'duelos_aereos_ganados_zona_area', 'duelos_aereos_ganados_zona_baja',
        'duelos_aereos_ganados_zona_media', 'duelos_aereos_ganados_zona_alta',
        'duelos_aereos_perdidos_zona_area', 'duelos_aereos_perdidos_zona_baja',
        'duelos_aereos_perdidos_zona_media', 'duelos_aereos_perdidos_zona_alta',
        'duelos_suelo_ganados_zona_area', 'duelos_suelo_ganados_zona_baja',
        'duelos_suelo_ganados_zona_media', 'duelos_suelo_ganados_zona_alta',
        'duelos_suelo_perdidos_zona_area', 'duelos_suelo_perdidos_zona_baja',
        'duelos_suelo_perdidos_zona_media', 'duelos_suelo_perdidos_zona_alta',
        
        # Intercepciones
        'intercepciones_zona_area', 'intercepciones_zona_baja',
        'intercepciones_zona_media', 'intercepciones_zona_alta',
        
        # Recuperaciones
        'recuperaciones_zona_area', 'recuperaciones_zona_baja',
        'recuperaciones_zona_media', 'recuperaciones_zona_alta',
        
        # Entradas
        'entradas_ganadas_zona_area', 'entradas_ganadas_zona_baja',
        'entradas_ganadas_zona_media', 'entradas_ganadas_zona_alta',
        'entradas_perdidas_zona_area', 'entradas_perdidas_zona_baja',
        'entradas_perdidas_zona_media', 'entradas_perdidas_zona_alta',
        
        # Faltas
        'faltas_zona_area', 'faltas_zona_baja', 'faltas_zona_media', 'faltas_zona_alta',
        'faltas_recibidas',
        
        # Tarjetas y amonestaciones
        'tarjetas_amarillas', 'tarjetas_rojas', 'amonestado',
        
        # Otros
        'fueras_de_juego'
    ]

    for resultado in ['ganadas', 'perdidas']:
        for zona in ['area', 'baja', 'media', 'alta']:
            metricas.append(f'entradas_{resultado}_zona_{zona}')

    for evento in ['faltas', 'intercepciones', 'recuperaciones']:
        for zona in ['area', 'baja', 'media', 'alta']:
            metricas.append(f'{evento}_zona_{zona}')

    for direccion in ['adelante', 'atras', 'horizontal']:
        for zona in ['inicio', 'creacion', 'finalizacion']:
            metricas.append(f'pases_{direccion}_{zona}')

    # Verificación final y resumen de métricas
    print("\ndf_eventos_metricas ha sido creado con éxito.")
    print(f"Dimensiones de df_events_league_all: {df_events_league_all.shape}")
    print(f"Dimensiones de df_eventos_metricas: {df_eventos_metricas.shape}")
    print(f"Número total de columnas en df_eventos_metricas: {len(df_eventos_metricas.columns)}")
    print("\nPrimeras 10 columnas de df_eventos_metricas:")
    print(df_eventos_metricas.columns[:10].tolist())
    print("...")
    print("\nÚltimas 10 columnas de df_eventos_metricas:")
    print(df_eventos_metricas.columns[-10:].tolist())

    # Verificar si alguna métrica tiene solo valores 0
    for metrica in metricas:
        if (df_eventos_metricas[metrica] == 0).all():
            print(f"\nAdvertencia: La métrica '{metrica}' tiene solo valores 0.")

    # Mostrar un resumen de las métricas
    print("\nResumen de las métricas:")
    print(df_eventos_metricas[metricas].sum())

    return df_eventos_metricas

# Uso de la función principal
if __name__ == "__main__":
    # Asegúrate de que df_events_league_all esté definido antes de esta línea
    df_eventos_metricas = main(df_events_league_all)

In [ ]:

# Paso 1: Crear columnas para goles a favor y goles en contra por equipo y partido
# Goles a favor (cuando es "Disparo" y "Gol")
df_eventos_metricas['gol_a_favor'] = np.where(
    (df_eventos_metricas['tipo_evento'] == 'Disparo') & (df_eventos_metricas['subtipo_evento'] == 'Gol'),
    1, 0
)

# Goles en contra (cuando es "Gol Concedido" o "Gol en Propia")
df_eventos_metricas['gol_en_contra'] = np.where(
    (df_eventos_metricas['tipo_evento'].isin(['Gol Concedido', 'Gol en Propia'])),
    1, 0
)

# Paso 2: Calcular goles acumulados por equipo y partido
# Goles a favor por equipo y partido
goles_a_favor = df_eventos_metricas.groupby(['match_id', 'team_id'])['gol_a_favor'].cumsum()

# Goles en contra por equipo y partido
goles_en_contra = df_eventos_metricas.groupby(['match_id', 'team_id'])['gol_en_contra'].cumsum()

# Guardar esos valores acumulados en el DataFrame
df_eventos_metricas['goles_a_favor'] = goles_a_favor
df_eventos_metricas['goles_en_contra'] = goles_en_contra

# Paso 3: Calcular el resultado acumulado de los equipos rivales
# Primero identificamos los equipos rivales dentro del mismo partido
df_eventos_metricas['team_rival'] = df_eventos_metricas.groupby('match_id')['team_id'].transform(lambda x: x.map(dict(zip(x.unique(), x.unique()[::-1]))))

# Goles a favor del rival (que son goles en contra de este equipo)
df_eventos_metricas['goles_rival'] = df_eventos_metricas.groupby(['match_id', 'team_rival'])['gol_a_favor'].cumsum()

# Paso 4: Calcular el estado del partido para el jugador (ganando, perdiendo o empatando)
# Si el equipo tiene más goles que el rival -> Ganando
# Si el equipo tiene menos goles que el rival -> Perdiendo
# Si los goles son iguales -> Empatando
df_eventos_metricas['resultado_partido'] = np.where(
    df_eventos_metricas['goles_a_favor'] > df_eventos_metricas['goles_rival'], 'Ganando',
    np.where(df_eventos_metricas['goles_a_favor'] < df_eventos_metricas['goles_rival'], 'Perdiendo', 'Empatando')
)

# Ver los primeros resultados
print(df_eventos_metricas[['match_id', 'player_id', 'team_id', 'gol_a_favor', 'gol_en_contra', 'goles_a_favor', 'goles_rival', 'resultado_partido']].head())


In [ ]:
def eliminar_duplicados_por_event_id(df):
    # Guardar número de filas original
    filas_originales = len(df)
    
    # Eliminar duplicados manteniendo la primera ocurrencia de cada event_id
    df_sin_duplicados = df.drop_duplicates(subset='event_id', keep='first')
    filas_finales = len(df_sin_duplicados)
    
    print(f"Se eliminaron {filas_originales - filas_finales} filas con event_id duplicados")
    print(f"El dataframe final tiene {filas_finales} filas con event_id únicos")
    
    return df_sin_duplicados

# Usar la función
df_eventos_metricas = eliminar_duplicados_por_event_id(df_eventos_metricas)

In [ ]:
# Guardar todos los datos en un archivo CSV
archivo_csv = 'eventos_metricas_alaves.csv'
df_eventos_metricas.to_csv(archivo_csv, index=False)

print(f"Archivo CSV '{archivo_csv}' creado exitosamente.")
print(f"Número de filas en el archivo: {len(df_eventos_metricas)}")
print(f"Número de columnas en el archivo: {len(df_eventos_metricas.columns)}")

# Mostrar información sobre las columnas
print("\nPrimeras 5 columnas del archivo:")
print(df_eventos_metricas.columns[:5].tolist())
print("...")
print("\nÚltimas 5 columnas del archivo:")
print(df_eventos_metricas.columns[-5:].tolist())

# Mostrar las primeras filas del DataFrame
print("\nPrimeras 3 filas del DataFrame:")
print(df_eventos_metricas.head(3).to_string())

# Información adicional sobre el archivo
print(f"\nTamaño del archivo: {df_eventos_metricas.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

In [ ]:
# Guardar los datos en un archivo Parquet
archivo_parquet = 'eventos_metricas_alaves.parquet'
df_eventos_metricas.to_parquet(archivo_parquet, index=False)

print(f"Archivo Parquet '{archivo_parquet}' creado exitosamente.")
print(f"Número de filas en el archivo: {len(df_eventos_metricas)}")
print(f"Número de columnas en el archivo: {len(df_eventos_metricas.columns)}")

# Mostrar información sobre las columnas
print("\nPrimeras 5 columnas del archivo:")
print(df_eventos_metricas.columns[:5].tolist())
print("...")
print("\nÚltimas 5 columnas del archivo:")
print(df_eventos_metricas.columns[-5:].tolist())

# Mostrar las primeras filas del DataFrame
print("\nPrimeras 3 filas del DataFrame:")
print(df_eventos_metricas.head(3).to_string())

# Información adicional sobre el archivo
import os
tamaño_archivo = os.path.getsize(archivo_parquet) / (1024 * 1024)  # Convertir a MB
print(f"\nTamaño del archivo: {tamaño_archivo:.2f} MB")

In [ ]:
for columna in df_eventos_metricas.columns:
    print(columna)

# Eliminar filas duplicadas
df_eventos_metricas = df_eventos_metricas.drop_duplicates()

# Verificar la cantidad de filas antes y después
print(f"Número de filas antes de eliminar duplicados: {len(df_eventos_metricas)}")
print(f"Número de filas después de eliminar duplicados: {len(df_eventos_metricas)}")

In [ ]:
# Definir las métricas que se van a agregar
metricas = [
        # Pases
        'pases_adelante', 'pases_atras', 'pases_horizontal',
        'pases_adelante_inicio', 'pases_adelante_creacion', 'pases_adelante_finalizacion',
        'pases_atras_inicio', 'pases_atras_creacion', 'pases_atras_finalizacion',
        'pases_horizontal_inicio', 'pases_horizontal_creacion', 'pases_horizontal_finalizacion',
        'pases_progresivos_inicio', 'pases_progresivos_creacion', 'pases_progresivos_finalizacion',
        'pases_fallidos_campo_propio', 'pases_fallidos_campo_contrario',
        'pases_exitosos_campo_propio', 'pases_exitosos_campo_contrario',
        'pases_largos_exitosos', 'pases_largos_fallidos',
        'centros_alejados', 'centros_linea_fondo',
        'cambios_orientacion_exitosos', 'cambios_orientacion_fallidos',
        
        # Recepciones
        'pases_recibidos_perfilado',
        
        # Conducciones
        'conducciones_inicio', 'conducciones_creacion', 'conducciones_finalizacion',
        'conducciones_carril_izquierdo', 'conducciones_carril_central', 'conducciones_carril_derecho',
        
        # Regates
        'regates_zona_inicio', 'regates_zona_creacion', 'regates_zona_finalizacion',
        
        # Disparos
        'disparos_a_puerta', 'disparos_fuera',
        
        # Despejes
        'despejes_zona_area', 'despejes_zona_baja', 'despejes_zona_media', 'despejes_zona_alta',
        
        # Duelos
        'duelos_aereos_ganados_zona_area', 'duelos_aereos_ganados_zona_baja',
        'duelos_aereos_ganados_zona_media', 'duelos_aereos_ganados_zona_alta',
        'duelos_aereos_perdidos_zona_area', 'duelos_aereos_perdidos_zona_baja',
        'duelos_aereos_perdidos_zona_media', 'duelos_aereos_perdidos_zona_alta',
        'duelos_suelo_ganados_zona_area', 'duelos_suelo_ganados_zona_baja',
        'duelos_suelo_ganados_zona_media', 'duelos_suelo_ganados_zona_alta',
        'duelos_suelo_perdidos_zona_area', 'duelos_suelo_perdidos_zona_baja',
        'duelos_suelo_perdidos_zona_media', 'duelos_suelo_perdidos_zona_alta',
        
        # Intercepciones
        'intercepciones_zona_area', 'intercepciones_zona_baja',
        'intercepciones_zona_media', 'intercepciones_zona_alta',
        
        # Recuperaciones
        'recuperaciones_zona_area', 'recuperaciones_zona_baja',
        'recuperaciones_zona_media', 'recuperaciones_zona_alta',
        
        # Entradas
        'entradas_ganadas_zona_area', 'entradas_ganadas_zona_baja',
        'entradas_ganadas_zona_media', 'entradas_ganadas_zona_alta',
        'entradas_perdidas_zona_area', 'entradas_perdidas_zona_baja',
        'entradas_perdidas_zona_media', 'entradas_perdidas_zona_alta',
        
        # Faltas
        'faltas_zona_area', 'faltas_zona_baja', 'faltas_zona_media', 'faltas_zona_alta',
        'faltas_recibidas',
        
        # Tarjetas y amonestaciones
        'tarjetas_amarillas', 'tarjetas_rojas',
        
        # Otros
        'fueras_de_juego'
    ]

# Paso 1: Incluir las columnas base en el DataFrame final, sin agregarlas (usando .first() para mantener su valor por grupo)
columnas_base = ['match_id', 'match', 'team_id', 'team_name', 'player_id', 'season_id',
                 'season_name', 'league_id', 'back_number', 'main_position', 'player']

df_acumulado = df_eventos_metricas.groupby(['match_id', 'player_id']).agg(
    {**{metric: 'sum' for metric in metricas},  # Sumar métricas
     'xg': 'sum',  # Sumar xg
     **{col: 'first' for col in columnas_base if col not in ['match_id', 'player_id']}}  # Mantener columnas base
).reset_index()

# Paso 2: Calcular las medias de 'xstart' y 'ystart', creando 'xmedia' y 'ymedia'
df_acumulado['xmedia'] = df_eventos_metricas.groupby(['match_id', 'player_id'])['xstart'].mean().values
df_acumulado['ymedia'] = df_eventos_metricas.groupby(['match_id', 'player_id'])['ystart'].mean().values

# Paso 3: Contar el número de "Exitoso" y "Fallido" por cada tramo del partido
# Obtener los tramos únicos del DataFrame
tramos = df_eventos_metricas['tramo_partido'].unique()

# Iterar sobre cada tramo y contar las veces que resultado es 'Exitoso' o 'Fallido'
for tramo in tramos:
    # Contar los éxitos
    exitoso_col = f'{tramo}_exitoso'
    df_exitosos = df_eventos_metricas[(df_eventos_metricas['tramo_partido'] == tramo) & (df_eventos_metricas['resultado'] == 'Exitoso')]
    df_acumulado[exitoso_col] = df_exitosos.groupby(['match_id', 'player_id']).size().reindex(df_acumulado.set_index(['match_id', 'player_id']).index, fill_value=0).values
    
    # Contar los fallidos
    fallido_col = f'{tramo}_fallido'
    df_fallidos = df_eventos_metricas[(df_eventos_metricas['tramo_partido'] == tramo) & (df_eventos_metricas['resultado'] == 'Fallido')]
    df_acumulado[fallido_col] = df_fallidos.groupby(['match_id', 'player_id']).size().reindex(df_acumulado.set_index(['match_id', 'player_id']).index, fill_value=0).values

# Paso 4: Reordenar las columnas para que las columnas base estén primero
# Obtener las demás columnas generadas, excluyendo las columnas base
otras_columnas = [col for col in df_acumulado.columns if col not in columnas_base]

# Crear un nuevo orden de columnas, con las columnas base primero
columnas_ordenadas = columnas_base + otras_columnas

# Reordenar el DataFrame para que siga el nuevo orden de columnas
df_acumulado = df_acumulado[columnas_ordenadas]

# El DataFrame df_acumulado ahora tiene las columnas base primero, seguidas de las otras columnas
print(df_acumulado.head())

# Guardar df_eventos_acumulados como un archivo CSV
df_acumulado.to_csv('eventos_datos_acumulados.csv', index=False)


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import Paragraph, SimpleDocTemplate, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.colors import Color
from reportlab.lib.utils import ImageReader
from PIL import Image

def create_pdf(output_filename):
    # Register a custom font (optional, you can use a default font if preferred)
    pdfmetrics.registerFont(TTFont('Vera', 'Vera.ttf'))

    # Create the PDF object
    doc = SimpleDocTemplate(output_filename, pagesize=letter)

    # Create styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'Title',
        parent=styles['Heading1'],
        fontName='Vera',
        fontSize=18,
        textColor=Color(0.1, 0.1, 0.1, alpha=0.8),  # Dark gray with some transparency
        spaceAfter=12
    )
    body_style = ParagraphStyle(
        'Body',
        parent=styles['BodyText'],
        fontName='Vera',
        fontSize=10,
        textColor=Color(0.1, 0.1, 0.1, alpha=0.8),  # Dark gray with some transparency
        spaceAfter=6
    )

    # Content for the PDF
    content = []
    
    content.append(Paragraph("Explicación de Métricas de Fútbol", title_style))
    content.append(Spacer(1, 12))
    
    metrics_explanations = [
        ("Cambios de Orientación", "Se consideran pases exitosos que cambian la dirección del juego, ya sea de un lado al otro del campo o de atrás hacia adelante."),
        ("Pases Largos", "Se dividen en dos categorías: de inicio (desde zonas 1-6) y de creación (desde zonas 7-12). Son pases que cubren una distancia significativa en el campo."),
        ("Pases Progresivos", "Pases que avanzan el balón hacia la portería rival, clasificados por la zona de inicio (inicio, creación, finalización)."),
        ("Conducciones", "Avances con el balón controlado, clasificados por la zona donde ocurren (inicio, creación, finalización)."),
        ("Tiros", "Se registran los tiros a puerta y fuera de puerta."),
        ("Pases Fallidos", "Se contabilizan los pases fallidos en campo propio y en campo contrario."),
        ("Intercepciones y Recuperaciones", "Clasificadas por zonas del campo (inicio, media, alta, área)."),
        ("Entradas, Faltas y Despejes", "También clasificados por zonas del campo."),
        ("Duelos", "Se registran duelos aéreos y en el suelo, ganados y perdidos, en diferentes zonas del campo."),
        ("Pases por Dirección y Zona", "Se clasifican los pases según su dirección (adelante, atrás, horizontal) y la zona de inicio (inicio, creación, finalización).")
    ]
    
    for title, explanation in metrics_explanations:
        content.append(Paragraph(title, title_style))
        content.append(Paragraph(explanation, body_style))
        content.append(Spacer(1, 12))

    # Build the PDF
    doc.build(content, onFirstPage=add_background, onLaterPages=add_background)

def add_background(canvas, doc):
    # Get the dimensions of the page
    page_width, page_height = letter

    # Open and resize the image to fit the page
    with Image.open('fondo_alaves.png') as img:
        img = img.resize((int(page_width), int(page_height)), Image.LANCZOS)
        img = ImageReader(img)

    # Draw the background image
    canvas.drawImage(img, 0, 0, width=page_width, height=page_height)

    # Add a semi-transparent white overlay to improve text readability
    canvas.setFillColor(Color(1, 1, 1, alpha=0.3))  # White with 30% opacity
    canvas.rect(0, 0, page_width, page_height, fill=True, stroke=False)

# Generate the PDF
create_pdf('metricas_futbol_explicacion.pdf')

print("PDF generated successfully: metricas_futbol_explicacion.pdf")

In [ ]:
resultado = df_eventos_metricas[df_eventos_metricas['season_id'] == 4019][['season_name', 'match']]
print(resultado)

In [ ]:
# Filtrar las filas donde tipo_evento es 'Balón Recibido'
df_balon_recibido = df_eventos_metricas[df_eventos_metricas['tipo_evento'] == 'Centro Recibido']

# Guardar el nuevo DataFrame en un archivo CSV
nombre_archivo = 'centro_recibido_eventos.csv'
df_balon_recibido.to_csv(nombre_archivo, index=False)

# Mostrar información sobre el nuevo DataFrame
print(f"Información del DataFrame 'Centro Recibido':")
print(df_balon_recibido.info())

print(f"\nSe ha creado el archivo CSV: {nombre_archivo}")
print(f"El archivo contiene {len(df_balon_recibido)} filas de eventos de 'Centro Recibido'.")

# Mostrar las primeras filas del nuevo DataFrame para verificar
print("\nPrimeras filas del DataFrame:")
print(df_balon_recibido.head())